# Import Model

In [1]:
from cameo import models
bigg_list = models.index_models_bigg()

/home/matinnu/anaconda3/envs/lab_rotation_3/lib/python3.7/site-packages/sympy/__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.singleton with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.singleton instead.
See https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()


In [2]:
# Find GEMS that starts with "Eschericia coli" and ends with "MG1655"
e_coli_mg1655 = bigg_list[bigg_list.organism.str.contains("^Escherichia coli.*MG1655$")]
e_coli_mg1655

,bigg_id,gene_count,reaction_count,organism,metabolite_count
0,e_coli_core,137,95,Escherichia coli str. K-12 substr. MG1655,72
2,iAF1260,1261,2382,Escherichia coli str. K-12 substr. MG1655,1668
3,iAF1260b,1261,2388,Escherichia coli str. K-12 substr. MG1655,1668
71,iJO1366,1367,2583,Escherichia coli str. K-12 substr. MG1655,1805
72,iJR904,904,1075,Escherichia coli str. K-12 substr. MG1655,761
76,iML1515,1516,2712,Escherichia coli str. K-12 substr. MG1655,1877


In [3]:
model_id = e_coli_mg1655.iloc[e_coli_mg1655.reaction_count.argmax()].bigg_id
model_id

'iML1515'

# Simulate Model

In [4]:
from cameo import load_model
model = load_model(model_id)
model

Using license file /home/matinnu/gurobi.lic
Academic license - for non-commercial use only


Name,iML1515
Memory address,0x07f20387f5a50
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [5]:
model.solver = 'gurobi'

In [6]:
from cameo import fba
%time fba_result = fba(model)

CPU times: user 391 ms, sys: 15.6 ms, total: 406 ms
Wall time: 613 ms


In [7]:
#fba_result.display_on_map("iJO1366.Central metabolism") #seems cameo not updated with escher yet

In [8]:
fba_result[model.reactions.BIOMASS_Ec_iML1515_core_75p37M]

0.8769972144269728

In [9]:
from escher import Builder

In [10]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    reaction_data=fba_result.fluxes,
)
builder

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': -0.022947509112696166, 'SHK3Dr': 0.3…

# Acetic Acid Producer Model
## Play Around, create list of knockouts

In [11]:
mutant1 = model.copy()

Read LP format model from file /tmp/tmpbaq5egql.lp
Reading time = 0.07 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Read LP format model from file /tmp/tmpsre7z5r9.lp
Reading time = 0.03 seconds
: 1877 rows, 5424 columns, 21150 nonzeros


In [12]:
import pandas as pd
import numpy as np
ko_list = pd.DataFrame(np.array(["ACKr", "ATPS4r", "ATPM", "FUM"]),
                   columns=['KO_GENES'])
ko_list

,KO_GENES
0,ACKr
1,ATPS4r
2,ATPM
3,FUM


In [13]:
ko_id = []
for i in ko_list.KO_GENES:
    #print(i)
    for counter, value in enumerate(mutant1.reactions):
        if i in value.id:
            #print(counter, value.id)
            target = [counter, value.id]
            #print(target)
            ko_id.append(target)
ko_id

[[80, 'ACKr'],
 [798, 'ATPS4rpp'],
 [1434, 'ATPM'],
 [195, 'FUM'],
 [671, 'SUCFUMtpp'],
 [875, 'FUMt2_2pp'],
 [876, 'FUMt2_3pp'],
 [939, 'FUMtex'],
 [2661, 'FUMt1pp']]

In [14]:
for i in ko_id:
    mutant1.reactions[i[0]].knock_out()
    print(model.reactions[i[0]])
    print("model: "+str(model.reactions[i[0]].bounds))
    print("mutant: "+str(mutant1.reactions[i[0]].bounds))

ACKr: ac_c + atp_c <=> actp_c + adp_c
model: (-1000.0, 1000.0)
mutant: (0, 0)
ATPS4rpp: adp_c + 4.0 h_p + pi_c <=> atp_c + h2o_c + 3.0 h_c
model: (-1000.0, 1000.0)
mutant: (0, 0)
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
model: (6.86, 1000.0)
mutant: (0, 0)
FUM: fum_c + h2o_c <=> mal__L_c
model: (-1000.0, 1000.0)
mutant: (0, 0)
SUCFUMtpp: fum_p + succ_c <=> fum_c + succ_p
model: (-1000.0, 1000.0)
mutant: (0, 0)
FUMt2_2pp: fum_p + 2.0 h_p --> fum_c + 2.0 h_c
model: (0.0, 1000.0)
mutant: (0, 0)
FUMt2_3pp: fum_p + 3.0 h_p --> fum_c + 3.0 h_c
model: (0.0, 1000.0)
mutant: (0, 0)
FUMtex: fum_e <=> fum_p
model: (-1000.0, 1000.0)
mutant: (0, 0)
FUMt1pp: fum_p <=> fum_c
model: (-1000.0, 1000.0)
mutant: (0, 0)


In [15]:
%time fba_knockout_result = fba(mutant1)
fba_knockout_result[mutant1.reactions.BIOMASS_Ec_iML1515_core_75p37M]

CPU times: user 250 ms, sys: 0 ns, total: 250 ms
Wall time: 296 ms


0.19672476125372415

In [16]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    reaction_data=fba_knockout_result.fluxes,
)
builder

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': 0.0053149128747918654, 'SHK3Dr': 0.0…

## Open List of Knocknout Genes

In [17]:
# waiting from Yixin

## Create Knockouts

## Simulate Knockouts

In [18]:
# Create Knockout - Acetic acid auxotroph


In [19]:
# Test the Knockout - auxotroph


In [20]:
# Test dFBA


In [21]:
# Create coculture model